In [3]:
import pandas as pd
df = pd.read_csv("c:/users/saini/Downloads/Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df.drop(columns=['RowNumber','CustomerId','Surname'],inplace=True)

In [6]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['Exited']),df['Exited'],random_state=42,test_size=0.2)

In [10]:
print(f'Count of Total Rows in Training Data Set : {X_train.shape}')
print(f'Count of Total Rows in Testing Data Set : {X_test.shape}')

Count of Total Rows in Training Data Set : (8000, 10)
Count of Total Rows in Testing Data Set : (2000, 10)


In [11]:
df.isnull().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [21]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

In [22]:
num_pipeline = Pipeline([
    ('num_imputation',SimpleImputer(strategy="mean")),
    ('feature_scaling',MinMaxScaler())
])

In [23]:
num_pipeline

Pipeline(steps=[('num_imputation', SimpleImputer()),
                ('feature_scaling', MinMaxScaler())])

In [24]:
from sklearn.preprocessing import OneHotEncoder

In [25]:
cat_pipeline = Pipeline([
    ('catg_imputation',SimpleImputer(fill_value="missing",strategy='constant')),
    ('one_hot_encoding',OneHotEncoder(sparse_output=False , handle_unknown='ignore'))
])

In [26]:
cat_pipeline

Pipeline(steps=[('catg_imputation',
                 SimpleImputer(fill_value='missing', strategy='constant')),
                ('one_hot_encoding',
                 OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

In [27]:
import numpy as np

In [40]:
num_cols = df.drop(columns=['Exited']).select_dtypes(include=np.number).columns.tolist()

In [41]:
num_cols

['CreditScore',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary']

In [42]:
cat_cols = df.drop(columns=['Exited']).select_dtypes(include='object').columns.tolist()

In [43]:
cat_cols

['Geography', 'Gender']

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


In [45]:
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier

In [46]:
preprocessor = ColumnTransformer([
    ('categorical',cat_pipeline,cat_cols),
    ('numerical',num_pipeline,num_cols)
])

In [47]:
pipe = Pipeline([
    ('preprocessor',preprocessor),
    ('estimator',RandomForestClassifier())
])

In [48]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('catg_imputation',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('one_hot_encoding',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['Geography', 'Gender']),
                                                 ('numerical',
                                                  Pipeline(steps=[('num_imputation',
                                                                   SimpleImputer()),
                                                                  ('feature_scaling',
                                                                   MinMaxScaler())]),
                                                  ['CreditScore', 'Age',
                                                   'Tenure', 'Balance',
                                                   'NumOfProducts', 'HasCrCard',
                                                   'IsActiveMember',
                                                   'EstimatedSalary'])])),
                ('estimator', RandomForestClassifier())])

In [49]:
 pipe.score(X_test,y_test)

0.865

In [51]:
y_pred = pipe.predict(X_test)

In [52]:
from sklearn.metrics import confusion_matrix

In [53]:
cm = confusion_matrix(y_pred,y_test)

In [54]:
cm

array([[1548,  211],
       [  59,  182]])